This file calculates the hazards counts(ramp lower than 20 is a poor ramp==hazard)

Map the counts to the sidewalks.

In [12]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd
import folium
import pandas as pd

In [13]:
#hazards counts
df = pd.read_csv('../data/Sidewalk_Hazards_1.csv')
hazard_df = df[['FID', 'Type', 'x', 'y']]
badramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
badramps_gdf = badramps_gdf.to_crs(sidewalks_gdf.crs)

In [14]:
badramps_gdf = badramps_gdf[badramps_gdf['score_1'] <= 20]

# Add a suffix to the FID column name in the badramps_gdf GeoDataFrame
badramps_gdf = badramps_gdf.rename(columns={'FID': 'FID_ramp'})

# Add a suffix to the FID column name in the hazard_df DataFrame
hazard_df = hazard_df.rename(columns={'FID': 'FID_hazard'})

hazard_gdf = gpd.GeoDataFrame(hazard_df, geometry=gpd.points_from_xy(hazard_df.x, hazard_df.y))
hazard_gdf.crs = 'EPSG:4326'
hazard_gdf = hazard_gdf.to_crs(sidewalks_gdf.crs)
hazard_gdf.geometry = hazard_gdf.geometry.buffer(5)
badramps_gdf.geometry = badramps_gdf.geometry.buffer(5)
#buffer area
print(hazard_gdf.head())

   FID_hazard                                   Type          x          y  \
0           1  Fixed pinch point <36” sidewalk width -71.069782  42.381454   
1           2     Trip hazards not due to tree roots -71.056617  42.295022   
2           3     Trip hazards not due to tree roots -71.069829  42.381396   
3           4     Trip hazards not due to tree roots -71.061326  42.296611   
4           5     Trip hazards not due to tree roots -71.061093  42.293839   

                                            geometry  
0  POLYGON ((772408.347 2964324.571, 772408.323 2...  
1  POLYGON ((776128.777 2932845.452, 776128.753 2...  
2  POLYGON ((772395.756 2964303.352, 772395.732 2...  
3  POLYGON ((774851.878 2933418.132, 774851.854 2...  
4  POLYGON ((774920.250 2932408.223, 774920.226 2...  


In [15]:
new_hazard_gdf = hazard_gdf.drop(['x', 'y'], axis=1)
new_hazard_gdf = new_hazard_gdf.rename(columns={'FID_hazard': 'FID'})


new_badramps_gdf = badramps_gdf[['FID_ramp', 'geometry','Failure_Ty']]
new_badramps_gdf = new_badramps_gdf.rename(columns={'FID_ramp': 'FID'})
new_badramps_gdf = new_badramps_gdf.rename(columns={"Failure_Ty": "Type"})
new_df_t = new_badramps_gdf
new_df_t.loc[new_df_t['Type'] == ' ', 'Type'] = 'Poor Ramp'
new_df_t['Type'].unique()
# new_df_t.info()
# new_badramps_gdf['Type'].isnull().sum()
# print(new_badramps_gdf.head())

array(['Concrete FailurePvt/Cswk Failure',
       'Concrete FailureDWP FailurePvt/Cswk Failure',
       'Concrete FailureDWP Failure', 'Missing/Depressed Dwy',
       'Concrete Failure', 'Poor Ramp'], dtype=object)

In [16]:
combined_gdf = pd.concat([new_badramps_gdf, new_hazard_gdf], ignore_index=True)
combined_gdf['Type'].unique()
# combined_gdf.to_csv('../data/combined_gdf.csv', index=False)
# print(combined_gdf.head())

array(['Concrete FailurePvt/Cswk Failure',
       'Concrete FailureDWP FailurePvt/Cswk Failure',
       'Concrete FailureDWP Failure', 'Missing/Depressed Dwy',
       'Concrete Failure', 'Poor Ramp',
       'Fixed pinch point <36” sidewalk width',
       'Trip hazards not due to tree roots',
       'Trip hazards due to tree roots'], dtype=object)

Where the attaching of hazards and sidewalks happens.

This version has the problem of types. Not using types in making maps.

In [17]:
# hazard_intersections = gpd.sjoin(sidewalks_gdf, combined_gdf, op='intersects', how='left')
# count_per_sidewalk = hazard_intersections.groupby('SWK_ID').size()
# hazard_intersections = hazard_intersections.join(count_per_sidewalk.rename('count'), on='SWK_ID')
# hazard_intersections['count'] = hazard_intersections['count'].fillna(0)
# hazard_intersections = hazard_intersections.dropna(subset=['geometry'])
# hazard_intersections = hazard_intersections.loc[:, ['SWK_ID', 'geometry', 'count','Type']]
# # Group the rows by SWK_ID and apply a function to each group
# grouped = hazard_intersections.groupby('SWK_ID').apply(lambda x: [s for s in x['Type'].astype(str).tolist() if s != 'nan'])
# hazard_intersections['Type_list'] = grouped.rename('Type').reset_index(drop=True)

c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 26563 entries, 0 to 23519
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   SWK_ID    26563 non-null  object  
 1   geometry  26563 non-null  geometry
 2   count     26563 non-null  int64   
 3   Type      9158 non-null   object  
dtypes: geometry(1), int64(1), object(2)
memory usage: 1.0+ MB
None


This comes from Billy. Not tested yet. Should solve the Type_list null problem.

In [ ]:
hazard_intersections = gpd.sjoin(sidewalks_gdf, combined_gdf, op='intersects', how='left')
count_per_sidewalk = hazard_intersections.groupby('SWK_ID').size()
hazard_intersections = hazard_intersections.join(count_per_sidewalk.rename('count'), on='SWK_ID')
hazard_intersections['count'] = hazard_intersections['count'].fillna(0)
hazard_intersections = hazard_intersections.dropna(subset=['geometry'])
hazard_intersections = hazard_intersections.loc[:, ['SWK_ID', 'geometry', 'count','Type']]
hazard_intersections['Type'] = hazard_intersections['Type'].astype('category')
index_to_drop = hazard_intersections.loc[hazard_intersections['SWK_ID'] == ' '].index
hazard_intersections.drop(index_to_drop, inplace = True)
# Group the rows by SWK_ID and create a new column 'Type_list'
grouped = hazard_intersections.groupby('SWK_ID')['Type'].apply(list).reset_index(name='Type_list')

# Merge the new 'Type_list' column with the original dataframe
hazard_intersections = hazard_intersections.merge(grouped, on='SWK_ID', how='left')

In [18]:
index_to_drop = hazard_intersections.loc[hazard_intersections['SWK_ID'] == ' '].index
hazard_intersections.drop(index_to_drop, inplace=True)
hazard_intersections[hazard_intersections['SWK_ID']==' ']

,SWK_ID,geometry,count,Type,Type_list


In [19]:
hazard_intersections = hazard_intersections.loc[:, ['SWK_ID','count', 'geometry']]
hazard_intersections.to_file('../data/compelete_scores.json',driver='GeoJSON')

Give the red sidewalks a new column with gpd

In [20]:

red_hazards_sidewalks = hazard_intersections[hazard_intersections['count'] >= 6 ]
red_hazards_sidewalks = red_hazards_sidewalks.rename(columns={'SWK_ID':'Red_ID'})
red_hazards_sidewalks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 743 entries, 37 to 23486
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Red_ID    743 non-null    object  
 1   count     743 non-null    int64   
 2   geometry  743 non-null    geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 23.2+ KB


Make the map of sidewalks rated by hazard counts. To add the Type_list, see the same cell in Billy's version

fields=['SWK_ID','count','Type_list']

In [21]:


m2 = folium.Map(location=[42.361145, -71.057083], zoom_start=13)



folium.Choropleth(
    geo_data=hazard_intersections,
    name='Hazard Counts Shapes',
    data=hazard_intersections,
    columns=['SWK_ID', 'count'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu_r',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Hazard Counts',
    highlight=True,
    show=False,
    bins=[0, 2, 4, 5, 6, 20]
).add_to(m2)




# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    hazard_intersections,
    name='Hazard Counts Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['SWK_ID','count'], labels=True, sticky=True),
    show=False
).add_to(m2)

# Add a layer control to the map
# folium.LayerControl().add_to(m2)

# Save the map as an HTML file
# m2.save('maps/All_hazard_count_xiao.html')

Add the layer of Poor sidewalks(Highlight)

In [22]:
# m2 = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

folium.Choropleth(
    geo_data=red_hazards_sidewalks,
    name='Poor sidewalk shapes',
    data=red_hazards_sidewalks,
    columns=['Red_ID', 'count'],
    key_on='properties.Red_ID',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Poor sidewalk',
    highlight=True,
    show=False,
).add_to(m2)

folium.GeoJson(
    red_hazards_sidewalks,
    name='Poor sidewalk scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['Red_ID','count'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 3, 'color':'red', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['Red_ID','count'], labels=True, sticky=True),
    show=False
).add_to(m2)

# Add a layer control to the map
folium.LayerControl().add_to(m2)

# Save the map as an HTML file
m2.save('maps/All_hazard_count_xiao.html')